# Test 5 channels (bf, dna, structure, cellseg, dnaseg)

## Load package and data

In [1]:
from upath import UPath as Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nbvv

from serotiny.io.image import image_loader
from cytodata_aics.io_utils import rescale_image

19-Oct-22 21:16:55 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


## Model

In [2]:
from serotiny.transforms.dataframe.transforms import split_dataframe
data_dir = "/home/aicsuser/cytodata-hackathon-base/data"

df = pd.read_csv(f"{data_dir}/mitocells.csv")
print(f'Number of cells: {len(df)}')
print(f'Number of columns: {len(df.columns)}')

Number of cells: 8179
Number of columns: 79


In [3]:
df["cell_stage"].value_counts()

M0               2000
M1M2             2000
M4M5             2000
M6M7_complete    1198
M3                981
Name: cell_stage, dtype: int64

In [4]:
import os
os.chdir("/home/aicsuser/cytodata-hackathon-base")

In [5]:
from datetime import datetime

# util to avoid referring to the same run unintentionally
now_str = lambda : datetime.now().strftime("%Y%m%d_%H%M%S")

# Load and train

In [15]:
run_name = f"teamx_bfdnastruct_dnaSeg_structSeg_vae_zproj_{now_str()}"
print(run_name)

!serotiny train \
    model=jan_fivechannel_vae_2d\
    data=jan_fivechannel_vae_dataloader_2d \
    mlflow.experiment_name=vae \
    mlflow.run_name={run_name} \
    trainer.gpus=[0] \
    trainer.max_epochs=2

# ++make_notebook=demo.ipynb

teamx_bfdnastruct_dnaSeg_structSeg_vae_zproj_20221019_222820
[2022-10-19 22:28:24,222][pytorch_lightning.utilities.seed][INFO] - Global seed set to 42
[2022-10-19 22:28:24,223][serotiny.ml_ops.ml_ops][INFO] - Instantiating datamodule
[2022-10-19 22:28:26,674][serotiny.ml_ops.ml_ops][INFO] - Instantiating trainer
[2022-10-19 22:28:26,727][pytorch_lightning.utilities.rank_zero][INFO] - GPU available: True, used: True
[2022-10-19 22:28:26,727][pytorch_lightning.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores
[2022-10-19 22:28:26,727][pytorch_lightning.utilities.rank_zero][INFO] - IPU available: False, using: 0 IPUs
[2022-10-19 22:28:26,727][pytorch_lightning.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
[2022-10-19 22:28:26,728][serotiny.ml_ops.ml_ops][INFO] - Instantiating model
[2022-10-19 22:28:26,758][serotiny.networks.basic_cnn.basic_cnn][INFO] - Determined 'compressed size': 41496 for CNN
[2022-10-19 22:28:27,719][serotiny.networks.basic_cnn.

In [9]:
!serotiny test \
    model=jan_fivechannel_vae_2d \
    data=jan_fivechannel_vae_dataloader_2d \
    mlflow.experiment_name=vae \
    mlflow.run_name={run_name} \
    trainer/callbacks=vae \
    ++force=True

[2022-10-19 21:04:05,576][pytorch_lightning.utilities.seed][INFO] - Global seed set to 42
[2022-10-19 21:04:05,577][serotiny.ml_ops.ml_ops][INFO] - Instantiating datamodule
[2022-10-19 21:04:07,812][serotiny.ml_ops.ml_ops][INFO] - Instantiating trainer
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
[2022-10-19 21:04:08,119][pytorch_lightning.utilities.rank_zero][INFO] - GPU available: True, used: False
[2022-10-19 21:04:08,120][pytorch_lightning.utilities.rank_zero][INFO] - TPU available: False, using: 0 TPU cores
[2022-10-19 21:04:08,120][pytorch_lightning.utilities.rank_zero][INFO] - IPU available: False, using: 0 IPUs
[2022-10-19 21:04:08,120][pytorch_lightning.utilities.rank_zero][INFO] - HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1812: